In [1]:
import numpy as np
import pandas as pd
import librosa

import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LayerNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score

In [3]:
# default hyperparameters
NEURONS = 300
HIDDEN_LAYERS = 3

def create_DNN(input_shape, neurons = NEURONS, hidden_layers = HIDDEN_LAYERS, learning_rate = 0.001, verbose=0):
    model = Sequential()

    model.add(Dense(neurons, input_dim=input_shape))
    model.add(Activation('relu'))

    for i in range(hidden_layers-1):
        model.add(Dense(neurons))
        model.add(Activation('sigmoid'))

    model.add(Dense(3))
    model.add(Activation('softmax'))

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) # categorical_crossentropy

    if verbose:
        model.summary()
    
    return model

In [ ]:
model = create_DNN(input_shape = x_train.shape[1], verbose=1)

In [ ]:
MODEL_SAVE_NAME = "AI_CUP_medical_sample_model"

train_results = model.fit(x_train, y_train, batch_size=None, epochs=100, 
                              callbacks=[EarlyStopping(monitor='val_loss', patience=10, mode='auto'),
                                         ModelCheckpoint(MODEL_SAVE_NAME+".h5", save_best_only=True)], 
                              validation_data=(x_val, y_val))

In [ ]:
y_pred = model.predict(training_df.iloc[:, :-1]).argmax(axis=1)
y_true = training_df['Disease category'] - 1

results_recall = recall_score(y_true, y_pred, average=None)
print("Training UAR(Unweighted Average Recall) :", results_recall.mean())
ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred)).plot(cmap='Blues')